# Question 1

In [1]:
import scrapy
import logging
from scrapy.crawler import CrawlerProcess

from lxml import etree
from lxml.cssselect import CSSSelector

from bs4 import BeautifulSoup
import csv
import json
import pandas as pd

In [2]:
data_dir_path='./'

In [3]:
import lxml.etree

import json

# receives the extract result from the spider and prints out the content
class ConsoleWriterPipeline(object):
    def open_spider(self, spider):
        None
    def close_spider(self, spider):
        None
    
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        print(line)
        return item
    
# receives the extract result from the spider and appends them into a JSON Line file, (each line is a json)
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open(data_dir_path+'result.json', 'w')

    def close_spider(self, spider):
        print('JSON File Generated')
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + '\n'
        self.file.write(line)
        return item

In [4]:
import unittest
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

class StoreSpider(scrapy.Spider):
    name = "store"
    start_urls = [
        'https://webscraper.io/test-sites/e-commerce/allinone/computers/tablets'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,                            # Default : Debug
        #'ITEM_PIPELINES': {'__main__.ConsoleWriterPipeline': 1} # Used for pipeline
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}
    }
    
    def parse(self, response):
        #Add code
        '''
        print (response)
        
        for item in response.css('div.caption'):
            yield {
                'pull-right price': item.css('span.text::text').get(),
                'description': item.css('span small::text').get(),
                'ratings': item.css('div.tags a.tag::text').getall(),
            }
        
        print ("--> Parse()")
        '''
        
        # use lxml to get decent HTML parsing speed
        soup = BeautifulSoup(response.text, 'html.parser')
        item_selector = soup.find_all('div', attrs={'class':'thumbnail'})
        i = 0
        tabletList = []
        for item in item_selector:
            #print (i, " ", item)
            #i = i+1
            tabletRow = {}
            price = item.find_all('h4', class_ = 'pull-right price')
            # Ref: https://blog.finxter.com/python-beautifulsoup-examples/
            for p in price:
                #tabletList.append(p.get_text())
                tabletRow['price'] = p.get_text()
                #print ("price: ", p.text.strip())
            #print ("price type: ", type(price), price[0])
            title = item.find_all('a', class_="title")
            for t in title:
                #tablet['product'] = t.get_text()
                #tabletList.append(t.get_text())
                tabletRow['product'] = t.get_text()
            #print ("title: ", title)
            desc = item.find_all('p', class_='description')
            for d in desc:
                #tablet['description'] = d.get_text()
                #tabletList.append(d.get_text())
                tabletRow['description'] = d.get_text()
            #print ("Description: ", desc)
            ratings = item.find_all('p', class_='pull-right')
            for r in ratings:
                #tablet['review'] = r.get_text()
                #tabletList.append(r.get_text())
                tabletRow['review'] = r.get_text()
            tabletList.append(tabletRow)                             
            yield {
                  'price': tabletRow['price'],
                  'product': tabletRow['product'],
                  'description': tabletRow['description'],
                  'review': tabletRow['review']
            }
        
# Function to run spider
def run_spider():
    configure_logging()
    runner = CrawlerRunner({"LOG_LEVEL": 'ERROR'})
    runner.crawl(StoreSpider)   
    d = runner.join()
    d.addBoth(lambda _: reactor.stop())

    reactor.run() # the script will block here until all crawling jobs are finished        

In [5]:
# Ref: https://github.com/scrapy/scrapy/issues/4826
run_spider()

2021-06-17 19:24:12 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}


JSON File Generated


In [6]:
#store_crawler_process = CrawlerProcess({
#    'USER_AGENT': 'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.1)'
#})

#store_crawler_process.crawl(StoreSpider)
#store_crawler_process.start()

In [7]:
# Ref: https://www.programmersought.com/article/84445530011/
# Fix for error: Extra data: line 2 column 1 (char 110)
l = []
file = open('result.json','r')
for line in file.readlines():
    l.append(json.loads(line))
print (l)

[{'price': '$69.99', 'product': 'Lenovo IdeaTab', 'description': '7" screen, Android', 'review': '7 reviews'}, {'price': '$88.99', 'product': 'IdeaTab A3500L', 'description': 'Black, 7" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2', 'review': '7 reviews'}, {'price': '$96.99', 'product': 'Acer Iconia', 'description': '7" screen, Android, 16GB', 'review': '7 reviews'}, {'price': '$97.99', 'product': 'Galaxy Tab 3', 'description': '7", 8GB, Wi-Fi, Android 4.2, White', 'review': '2 reviews'}, {'price': '$99.99', 'product': 'Iconia B1-730HD', 'description': 'Black, 7", 1.6GHz Dual-Core, 8GB, Android 4.4', 'review': '1 reviews'}, {'price': '$101.99', 'product': 'Memo Pad HD 7', 'description': 'IPS, Dual-Core 1.2GHz, 8GB, Android 4.3', 'review': '10 reviews'}, {'price': '$102.99', 'product': 'Asus MeMO Pad', 'description': '7" screen, Android, 8GB', 'review': '14 reviews'}, {'price': '$103.99', 'product': 'Amazon Kindle', 'description': '6" screen, wifi', 'review': '3 reviews'}, {'price': '$107.99

In [8]:
df = pd.DataFrame(l)
print (df['review'].str.replace(" reviews", "").astype('int64'))

df.review = df['review'].str.replace(" reviews", "").astype('int64')
df

0      7
1      7
2      7
3      2
4      1
5     10
6     14
7      3
8     14
9     13
10    11
11     9
12     8
13     1
14    14
15     7
16    12
17     9
18     8
19     6
20     7
Name: review, dtype: int64


,price,product,description,review
0,$69.99,Lenovo IdeaTab,"7"" screen, Android",7
1,$88.99,IdeaTab A3500L,"Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2",7
2,$96.99,Acer Iconia,"7"" screen, Android, 16GB",7
3,$97.99,Galaxy Tab 3,"7"", 8GB, Wi-Fi, Android 4.2, White",2
4,$99.99,Iconia B1-730HD,"Black, 7"", 1.6GHz Dual-Core, 8GB, Android 4.4",1
5,$101.99,Memo Pad HD 7,"IPS, Dual-Core 1.2GHz, 8GB, Android 4.3",10
6,$102.99,Asus MeMO Pad,"7"" screen, Android, 8GB",14
7,$103.99,Amazon Kindle,"6"" screen, wifi",3
8,$107.99,Galaxy Tab 3,"7"", 8GB, Wi-Fi, Android 4.2, Yellow",14
9,$121.99,IdeaTab A8-50,"Blue, 8"" IPS, Quad-Core 1.3GHz, 16GB, Android 4.2",13


In [9]:
def SearchbyReviewGreaterthan(review):
    return (df.loc[df['review']>review])
    #print (type(df['review'].loc[df['review']>review]))

In [10]:
reviews = SearchbyReviewGreaterthan(5)
print (reviews)

      price           product  \
0    $69.99    Lenovo IdeaTab   
1    $88.99    IdeaTab A3500L   
2    $96.99       Acer Iconia   
5   $101.99     Memo Pad HD 7   
6   $102.99     Asus MeMO Pad   
8   $107.99      Galaxy Tab 3   
9   $121.99     IdeaTab A8-50   
10  $130.99        MeMO Pad 7   
11  $148.99   IdeaTab A3500-H   
12  $172.99     IdeaTab S5000   
14  $251.99        Galaxy Tab   
15  $320.99   MeMo PAD FHD 10   
16  $399.99       Galaxy Note   
17  $489.99       Galaxy Note   
18  $537.99  iPad Mini Retina   
19  $587.99  Galaxy Note 10.1   
20  $603.99    Apple iPad Air   

                                          description  review  
0                                  7" screen, Android       7  
1   Black, 7" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2       7  
2                            7" screen, Android, 16GB       7  
5             IPS, Dual-Core 1.2GHz, 8GB, Android 4.3      10  
6                             7" screen, Android, 8GB      14  
8                 7",